# Python module

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SQLContext, Row

# Data 파일 열기

In [ ]:
spark = SparkSession.builder.getOrCreate()

* header를 같이 불러오려면 header=True를 반드시 써줘야함

In [ ]:
data = spark.read.csv("./Crimes_Chicago.csv", header=True)

In [ ]:
# data.printSchema()

In [ ]:
# data.show()

In [ ]:
# type(data)

# 결측지 제거

In [ ]:
chicago = data.alias("chicago")

* 특정 열에 결측지 있는 행 제거하는 구문

(참조) https://stackoverflow.com/questions/35477472/difference-between-na-drop-and-filtercol-isnotnull-apache-spark

In [ ]:
chicago = data.na.drop(subset =["Longitude"])

* 행 개수 count

In [ ]:
# chicago.count()

In [ ]:
# data.count()

* 임시저장..?

In [ ]:
chicago.cache()

* 이 부분에서 에러 발생했는데 pyspark 버전 문제 때문이었음


cmd에 pip install pyspark==2.3.0

(참조) https://stackoverflow.com/questions/54546513/py4jjavaerror-an-error-occurred-while-calling

이라고 생각했지만 놀랍게도 메모리 터져서 안됨
왜 터지고 난리죠..?


In [ ]:
# pd_chicago = chicago.toPandas()

# 년도별 연습

In [ ]:
# chicago.filter('Year==2001').show()

In [ ]:
# dd = SQLContext.sql("SELECT * from chicago WHERE Year <> '2011'")

* view 만들어서 sql문 사용 가능하도록

In [ ]:
chicago.createOrReplaceTempView("CHICAGO")

In [ ]:
# spark.sql("SELECT * FROM CHICAGO WHERE Year = '2001'").show()

In [ ]:
# chicago.filter(chicago["Year"] == '2001').show()

# 년도별 카운트

In [ ]:
year_count = spark.sql("SELECT Year, count(Year) AS Count FROM CHICAGO GROUP BY Year ORDER BY Year")

In [ ]:
year_count.show()

In [ ]:
type(year_count)

* 찾아보니 Plot 쓰려면 pyspark.sql.dataframe인 상태로는 못하고 로컬로 가져와야 해서

collect()

toPandas()


사용해야 한다고 함

근데 또 너무 큰거 로컬로 가져오려고 하면 메모리가 터져버리니 아이러니..

In [ ]:
pd_year_count = year_count.toPandas()

In [ ]:
plt.figure(figsize=(12,6), dpi=80)
plt.xlabel('Year', size = 15)
plt.ylabel('Count',  size = 15)
plt.title('Chicago Data Count by Year', size = 17)
plt.bar(pd_year_count['Year'],pd_year_count['Count'], color = "#ff6f61")
plt.show()

# 년도별 데이터 분류

* pySpark dataframe column to list in python (1)

아래 방법 쓰려고 했는데 ROW자체가 요소로 들어감

In [ ]:
pyear_lst = year_count.select('Year').collect()

In [ ]:
# type(year_lst[0])

* pySpark dataframe column to list in python (2)

이 방법은 pandas dataframe -> list로 바꾸는 방식임

In [ ]:
year_lst = list(year_count.select('Year').toPandas()['Year'])

* sql문에서 python 변수 사용하는 방법

(참조) https://stackoverflow.com/questions/44582450/how-to-pass-variables-in-spark-sql-using-python

In [ ]:
i=5
tmp = spark.sql("SELECT * FROM CHICAGO WHERE Year = {}".format(year_lst[i]))

* toPandas 넣는 순간 억겁의 시간을 기다리게 되어서 멈춤

In [ ]:
# yearlst = []
# for i in range(len(year_lst)):
#     print(year_lst[i])
#     tmp = spark.sql("SELECT * FROM CHICAGO WHERE Year = {}".format(year_lst[i]))
#     print(i," finish")
#     yearlst.append(tmp.toPandas())
#     print("*** to pandas ***")

* 그래서 우선 Pyspark 형태로 list 저장함

차라리 이걸 각각 csv로 저장한다음에 pandas dataframe으로 csv 읽어서 plot 그리거나

계산할거면 pyspark 쓴 후에 csv 저장하고..이런식으로 하는게 더 빠를 수도 

(년도 한두개만 toPandas는 기다릴만 한듯)

In [ ]:
yearlst = []
for i in range(len(year_lst)):
#     print(year_lst[i])
    tmp = spark.sql("SELECT * FROM CHICAGO WHERE Year = {}".format(year_lst[i]))
    yearlst.append(tmp)

In [ ]:
yearlst[0]

# 아니면 SQL에서 바로 계산해버리기

1. 월별 평균치 통계

* 를 위해서 Month, Day 열을 추가해야함

In [ ]:
tmp = chicago

In [ ]:
tmp = spark.sql("SELECT * FROM CHICAGO )

In [ ]:
tmp = spark.sql("SELECT *, SUBSTR(Date,0,2) AS Month, SUV AS Day FROM CHICAGO")

In [ ]:
tmp.show()